In [1]:
%run qsu.ipynb
import numpy as np
import sympy
import networkx as nx
from qecsim import paulitools as pt 
from qecsim.models.generic import DepolarizingErrorModel
from qecsim.models.toric import ToricCode
import nbimporter
from typing import List, Tuple
import hypergraph_prod_code as hpc

In [84]:
def belief_prop(H: np.array, s: np.array, p: float, max_iter: int) -> Tuple:
    """ 
    Belief Propagation Algorithm for Decoding LDPC Codes

    Parameters:
    -----------
    H - parity-check matrix corresponding to either X or Z checks
    s - Error syndrome
    p - Channel error rate for chosen noise channel
    max_iter - Maximum number of iterations to run BP algorithm for
    """
    data_to_parity = np.zeros((len(H[0]),len(H)), dtype=float)
    parity_to_data = np.zeros((len(H), len(H[0])), dtype=float)
    H_tanner_graph = hpc.parity_check_mat_to_tanner(H)
    
    # Channel Log Likelihood Ratio
    p_l = np.log((1 - p)/p)
    
    P_1 = np.zeros((len(H[0]),), dtype=float)
    e_BP = np.zeros((len(H[0]),), dtype=float)

    # (1) Initialization
    for edge in H_tanner_graph.edges:
        data_node_num = int(edge[0][1:])
        parity_node_num = int(edge[1][1:])
        data_to_parity[data_node_num][parity_node_num] = p_l 

    for iter in range(1, max_iter + 1):
        # Scaling Factor
        a = 1 - 2**(-1 * iter)

        # (2) Parity to Data Messages
        for edge in H_tanner_graph.edges:
            parity_node_num = int(edge[1][1:])
            data_node_num = int(edge[0][1:])

            # Get list of neighbors of current parity_node set minus the current data node
            V = list(nx.neighbors(H_tanner_graph, edge[1]))
            V.remove(edge[0])

            # Get messages from elements of V to current parity node
            data_to_par_msgs = [data_to_parity[int(v[1:])][parity_node_num] for v in V]
            w = np.min([np.abs(msg) for msg in data_to_par_msgs])
            parity_to_data[parity_node_num][data_node_num] = ((-1) ** int(s[parity_node_num])) * a * np.prod(np.sign(data_to_par_msgs)) * w 

        # (3) Data to Parity Messages
        for edge in H_tanner_graph.edges:
            data_node_num = int(edge[0][1:])
            parity_node_num = int(edge[1][1:])

            # Get list of neighbors of current data node set minus the current parity node
            U = list(nx.neighbors(H_tanner_graph, edge[0]))
            U.remove(edge[1])

            # Get messages from elements of U to current data node
            par_to_data_msgs = [parity_to_data[int(u[1:])][data_node_num] for u in U]
            data_to_parity[data_node_num][parity_node_num] = p_l + np.sum(par_to_data_msgs)

        """
        # Hard Decision 
        for data_node in [node for node in H_tanner_graph.nodes if node[0] == 'v']:
            data_node_num = int(data_node[1:])

            # Get list of neighbors of current data node 
            U = list(nx.neighbors(H_tanner_graph, data_node))

            # Get messages from elements of U to current data node
            par_to_data_msgs = [parity_to_data[int(u[1:])][data_node_num] for u in U]
            P_1[data_node_num] = p_l + np.sum(par_to_data_msgs)
            e_BP[data_node_num] = -1 * np.sign(P_1[data_node_num])
        #print(e_BP)
        """

        # Hard Decision
        for edge in H_tanner_graph.edges:
            data_node_num = int(edge[0][1:])
            parity_node_num = int(edge[1][1:])

            # Get list of neighbors of current data node
            U = list(nx.neighbors(H_tanner_graph, edge[0]))

            par_to_data_msgs = [parity_to_data[int(u[1:])][data_node_num] for u in U]
            P_1[data_node_num] = p_l + np.sum(par_to_data_msgs)
            e_BP[data_node_num] = -1 * np.sign(P_1[data_node_num])
        
        
        # (4) Termination Check
        e_BP = e_BP * (e_BP > 0)
        print(e_BP)
        if (np.array_equal(np.dot(H, e_BP), s)):
            return True, e_BP, P_1 

    return False, e_BP, P_1

# Turn to Ordered Statistics Decoding if BP fails to converge
def OSD_0(H: np.array, P_1: np.array, s: np.array) -> np.array:
    """ 
    The Ordered Statistics Decoding (OSD) Zero algorithm is a post-processing 
    algorithm utilized when BP fails to converge 

    Parameters:
    -----------
    H - parity check matrix
    P_1 - BP soft decision vector
    s - Error syndrome 

    Returns:
    --------
    Error string
    """

    # Get the rank of the parity check matrix
    H_rank = np.linalg.matrix_rank(H)

    # Maintain a mapping between bit positions and elements of the BP soft-decision vector
    P_1_sorted_pos = np.argsort(P_1, kind='stable')
    P_1_sorted = [P_1[i] for i in P_1_sorted_pos]

    # Rearrange columns of H to match the reordered soft-decision vector
    idx = np.empty_like(P_1_sorted_pos)
    H[:] = H[:, idx]

    # Select first RANK(H) linearly independent columns of above rearrangement
    _, inds = sympy.Matrix(H).rref()
    H_S = np.vstack((H[:][inds[i]] for i in range(0, H_rank))).T

    # Calculate the OSD-0 solution on the basis-bits
    e_S = np.linalg.inv(H_S) * s
    e_ST = np.hstack((e_S, np.zeros((len(H[0]) - H_rank,))))

    # Map the OSD-0 solution to the original bit-ordering
    e_OSD = np.zeros((len(H[0]),))
    for i in range(len(P_1_sorted_pos)):
        e_OSD[P_1_sorted_pos[i]] = e_ST[i]

    return e_OSD

"""
# Turn to Ordered Statistics Decoding if BP fails to converge
def OSD_0(H: np.array, P_1: np.array, s: np.array) -> np.array:
    #S = np.array()

    # Get the rank of the parity check matrix
    H_rank = rank(matrix(H))
    print(H_rank)

    # Maintain a mapping between elements of the BP soft-decision vector and bit positions 
    pos = [i for i in range(len(P_1))]
    P_1_dict = {i:p for (i,p) in zip(P_1, pos)}
    P_1_sorted = np.sort(P_1)
    P_1_sorted_pos = [P_1_dict[p] for p in P_1_sorted]
    
    # Rearrange columns of H to match the reordered soft-decision vector
    idx = np.empty_like(P_1_sorted_pos)
    H[:] = H[:, idx]

    # Select first RANK(H) linearly independent columns of above rearrangement
    _, inds = sympy.Matrix(H).rref()
    H_S = np.vstack((H[:][inds[i]] for i in range(0, H_rank))).T

    # Calculate the OSD-0 solution on the basis-bits
    e_S = np.linalg.inv(H_S) * s
    e_ST = np.hstack((e_S, np.zeros((len(H[0]) - H_rank,))))

    # Map the OSD-0 solution to the original bit-ordering
    e_OSD = np.zeros((len(H[0]),))
    for i in range(len(P_1_sorted_pos)):
        e_OSD[P_1_sorted_pos[i]] = e_ST[i]

    return e_OSD
"""

def OSD_0_Plus(H: np.array, P_1: np.array, s: np.array, e_T: np.array) -> np.array:
    """ 
    The Higher Order OSD algorithm is a post-processing 
    algorithm utilized when BP fails to converge, building 
    on OSD-0 Algorithm

    Parameters:
    -----------
    H - parity check matrix
    P_1 - BP soft decision vector
    s - Error syndrome 
    e_T - Choice of error bits on remaining bits that aren't getting flipped with high enough probability

    Returns:
    --------
    Error string 
    """
    GF = galois.GF(2)
    S = np.array()

    # Get the rank of the parity check matrix
    H_rank = rank(matrix(H))

    # Maintain a mapping between elements of the BP soft-decision vector and bit positions 
    P_1_dict = {p:i for p in P_1 for i in range(len(P_1))}
    P_1_sorted = np.sort(P_1)
    P_1_sorted_pos = [P_1_dict[p] for p in P_1_sorted]
    
    # Rearrange columns of H to match the reordered soft-decision vector
    idx = np.empty_like(P_1_sorted_pos)
    H[:] = H[:, idx]

    # Select first RANK(H) linearly independent columns of above rearrangement
    _, inds = sympy.Matrix(H).rref()
    H_S = np.vstack((H[:][inds[i]] for i in range(0, H_rank))).T
    H_T = np.vstack(H[:][inds[i]] for i in range (H_rank, len(H[0])))

    # Calculate the OSD-0 solution on the basis-bits
    e_S = np.linalg.inv(H_S) * s
    e_ST_1 = GF(np.linalg.inv(H_S) * e_S + np.linalg.inv(H_S) * H_T * e_T)
    e_ST_2 = GF(e_T)
    e_ST = np.hstack((e_ST_1, e_ST_2))

    # Map the OSD-0 solution to the original bit-ordering
    e_OSD = np.zeros((len(H[0]),))
    for i in range(len(P_1_sorted_pos)):
        e_OSD[P_1_sorted_pos[i]] = e_ST[i]

    return e_OSD

In [85]:
# initialize models
dim = 3
my_code = ToricCode(dim,dim)
my_error_model = DepolarizingErrorModel()

In [100]:
# Set physical error probability to 10%
error_probability = 0.1
# Seed random number generator for repeatability
rng = np.random.default_rng(27)

# Error: random error based on error probability
error = my_error_model.generate(my_code, error_probability, rng)
print(error)
print(('error:\n{}'.format(my_code.new_pauli(error))))

[0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
error:
┼─·─┼─·─┼─·
Y   ·   ·  
┼─·─┼─X─┼─·
·   X   ·  
┼─·─┼─·─┼─·
·   ·   ·  


In [101]:
# Syndrome: Stabilizers that do not commute with the error
syndrome = pt.bsp(error, my_code.stabilizers.T)
print(syndrome)
print(('syndrome:\n{}'.format(my_code.ascii_art(syndrome))))

[1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0]
syndrome:
X───┼───┼──
│ Z │ Z │ Z
X───┼───┼──
│ Z │   │  
┼───┼───┼──
│   │   │  


In [102]:
Z_stabs = (my_code.stabilizers[:dim ** 2])[:, 2 * dim ** 2:]
X_stabs = (my_code.stabilizers[dim ** 2:])[:, :2 * dim ** 2]
Z_error = error[2 * dim**2:]
X_error = error[:2 * dim**2]
print("Z_error: " + str(Z_error))
print("X_error: " + str(X_error))

Z_error: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
X_error: [0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0]


In [103]:
Z_syndrome = np.mod(X_stabs @ Z_error,2)
print("Z_syndrome: " + str(Z_syndrome))
X_syndrome = np.mod(Z_stabs @ X_error, 2)
print("X_syndrome: " + str(X_syndrome))

Z_syndrome: [1 0 0 0 0 0 1 0 0]
X_syndrome: [1 1 1 1 0 0 0 0 0]


In [104]:
converged_Z, e_BP_Z, P_1_Z = belief_prop(X_stabs, Z_syndrome, 0.1, 2 * dim ** 2)
converged_X, e_BP_X, P_1_X = belief_prop(Z_stabs, X_syndrome, 0.1, 2 * dim ** 2)

[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
[-0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0.]
[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
[-0. -0. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
[-0. -0. -0.  1. -0. -0. -0. -0. -0. -0. -0.  1. -0. -0. -0. -0. -0. -0.]


In [105]:
print(converged_Z, e_BP_Z, P_1_Z)
print('\n')
print(converged_X, e_BP_X, P_1_X)

True [-0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0.] [ 3.84514301  3.84514301  3.84514301  3.84514301  3.84514301  3.84514301
  5.49306144  7.14097988  5.49306144 -2.74653072  3.84514301  3.84514301
  2.19722458  5.49306144  5.49306144  2.19722458  5.49306144  5.49306144]


True [-0. -0. -0.  1. -0. -0. -0. -0. -0. -0. -0.  1. -0. -0. -0. -0. -0. -0.] [ 3.63915321  3.63915321  3.63915321 -1.64791843  2.19722458  2.19722458
  2.19722458  4.60043896  4.60043896  1.23593882  1.23593882 -0.2059898
  2.19722458  2.19722458  4.60043896  4.60043896  4.60043896  4.60043896]


In [88]:
e_OSD = OSD_0(X_stabs, P_1, Z_syndrome)

IndexError: index 4612130128693887755 is out of bounds for axis 1 with size 18